In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#각도 feature 추출 함수
import math
def dist(v) :
    return math.sqrt(v[0]**2+v[1]**2)

def angleBetweenTwo(v1, v2):

    distA = dist(v1)
    distB = dist(v2)

    ip = v1[0]*v2[0] + v1[1]*v2[1]
    ip2= distA* distB
    cost = ip/ ip2
    return(cost)

#[1] 특징점 인식을 위한 라이브러리, 이미지 디렉토리 변수에 저장

In [ ]:
import numpy as np
import cv2
import dlib

from google.colab.patches import cv2_imshow
import os

# 얼굴 검출을 위한 dlib 얼굴 인식 모델
detector = dlib.get_frontal_face_detector()

# 얼굴 특징점 검출을 위한 dlib 얼굴 인식 모델
faceCascade = cv2. CascadeClassifier ('/content/drive/MyDrive/Colab Notebooks/haarcascade_frontalface_default.txt')
predictor = dlib.shape_predictor ("/content/drive/MyDrive/Colab Notebooks/shape_predictor_68_face_landmarks.dat")

# 이미지가 저장된 디렉토리 경로
img_dir_path = '/content/drive/MyDrive/matlab 공모전/data_file/train_test_data'

## 이미지 데이터 개수 확인


In [ ]:
#총 데이터 개수:
total_num=0
img_dir_path = '/content/drive/MyDrive/matlab 공모전/data_file/train_test_data'
for filename in os.listdir(img_dir_path):
    if filename.endswith(".jpg"):
        # 이미지 파일 읽기
        total_num+=1
total_num

15059

## 파일 이름 내림차순 정렬
feature 데이터프레임에 입력되는 순서 = 파일 이름 내림차순

In [ ]:
#정렬한 파일이름
A = os.listdir(img_dir_path)
sorted_fname = sorted(A)

In [ ]:
t_path='/content/drive/MyDrive/matlab 공모전/data_file/train_test_data'
tlist = os.listdir(t_path)
t_sorted=sorted(tlist)

In [ ]:
#1.인식된 얼굴 사진이라면, train_test_df에 feature 저장
import pandas as pd
t=0
count=0
train_test_df = pd.DataFrame(index = range(total_num), columns = ('filename','leye_ratio','reye_ratio','eye_angle', 'nose_ratio','chin_sharp'))

#[2] 모든 이미지에 대해 특징점 인식후, feature값 추출

In [ ]:
#전체 데이터 feature 만들기
total_fin = 0
for t in range(len(t_sorted)):
      filename = t_sorted[t]
      # 이미지 파일 읽기
      img_path = os.path.join(img_dir_path, filename)
      img = cv2.imread(img_path)

      # 이미지를 그레이스케일로 변환
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

      # 얼굴 및 68개 얼굴 특징점 검출
      faces = detector(gray, 1)
      #얼굴에 사각형을 친다.
      for box in faces:
          x1 = box.left()
          y1 = box.top()
          x2 = box.right()
          y2 = box.bottom()
          dlib_rect = dlib.rectangle(x1, y1, x2, y2)
          #landmarks_display : 68개 얼굴 특징점 위치에 접근
          landmarks = np.array([[p.x, p.y] for p in predictor(img, dlib_rect).parts()])
          landmarks_display = landmarks[0:68]

          #feature 추출에 필요한 눈 특징점
          lefttop = [0,0]
          leftbot = [0,0]
          righttop = [0,0]
          rightbot = [0,0]

          leftle = np.squeeze(np.asarray(landmarks_display[36]))
          leftri = np.squeeze(np.asarray(landmarks_display[39]))
          lefttop1 = np.squeeze(np.asarray(landmarks_display[37]))
          lefttop2 = np.squeeze(np.asarray(landmarks_display[38]))
          leftbot1 = np.squeeze(np.asarray(landmarks_display[41]))
          leftbot2 = np.squeeze(np.asarray(landmarks_display[40]))

          rightle = np.squeeze(np.asarray(landmarks_display[42]))
          rightri = np.squeeze(np.asarray(landmarks_display[45]))
          righttop1 = np.squeeze(np.asarray(landmarks_display[43]))
          righttop2 = np.squeeze(np.asarray(landmarks_display[44]))
          rightbot1 = np.squeeze(np.asarray(landmarks_display[47]))
          rightbot2 = np.squeeze(np.asarray(landmarks_display[46]))

          #중앙 점을 저장함.
          lefttop[0] = (lefttop1[0]+lefttop2[0])/2
          lefttop[1] =(lefttop1[1]+lefttop2[1])/2
          leftbot[0] = (leftbot1[0]+leftbot2[0])/2
          leftbot[1] = (leftbot1[1]+leftbot2[1])/2

          righttop[0] = (lefttop1[0]+lefttop2[0])/2
          righttop[1] =(lefttop1[1]+lefttop2[1])/2
          rightbot[0] = (leftbot1[0]+leftbot2[0])/2
          rightbot[1] = (leftbot1[1]+leftbot2[1])/2


          #양눈 높이/너비 비율
          leye_wid=math.sqrt((leftle[0]-leftri[0])**2+(leftle[1]-leftri[1])**2)
          leye_hgt = math.sqrt((lefttop[0]-rightbot[0])**2+(lefttop[1]-leftbot[1])**2)

          reye_wid =math.sqrt((rightle[0]-rightri[0])**2+(rightle[1]-rightri[1])**2)
          reye_hgt =math.sqrt((righttop[0]-rightbot[0])**2+(righttop[1]-rightbot[1])**2)

          leye_ratio = leye_hgt/leye_wid
          reye_ratio = reye_hgt/reye_wid

          #양눈 각도
          c=  np.squeeze(np.asarray(landmarks_display[42]))
          d =  np.squeeze(np.asarray(landmarks_display[45]))
          e = np.squeeze(np.asarray(landmarks_display[36]))
          f = np.squeeze(np.asarray(landmarks_display[39]))

          reye_vec = d-c
          leye_vec = e-f
          eye_angle = angleBetweenTwo(reye_vec, leye_vec)

          #코 너비/길이 (얇은가, 넓은가?)
          nosele = np.squeeze(np.asarray(landmarks_display[31]))
          noseri = np.squeeze(np.asarray(landmarks_display[35]))
          nosetop = np.squeeze(np.asarray(landmarks_display[27]))
          nosebot = np.squeeze(np.asarray(landmarks_display[33]))

          nose_wid=math.sqrt((nosele[0]-noseri[0])**2+(nosele[1]-noseri[1])**2)
          nose_hgt = math.sqrt((nosetop[0]-nosebot[0])**2+(nosetop[1]-nosebot[1])**2)
          nose_ratio = nose_wid/nose_hgt

          #좁아지는각도
          left =  np.squeeze(np.asarray(landmarks_display[5]))
          chinbot =  np.squeeze(np.asarray(landmarks_display[8]))
          right =  np.squeeze(np.asarray(landmarks_display[11]))


          l = chinbot-left
          r = chinbot-right

          chin_sharp =  angleBetweenTwo(l, r)
          train_test_df.iloc[t][0] = filename
          train_test_df.iloc[t][1] = leye_ratio
          train_test_df.iloc[t][2] = reye_ratio
          train_test_df.iloc[t][3] = eye_angle
          train_test_df.iloc[t][4] = nose_ratio
          train_test_df.iloc[t][5] = chin_sharp
          total_fin +=1
          t+=1

In [ ]:
#인식된 파일 개수
total_fin

15007

In [ ]:
#인식안된 파일의 인덱스
train_test_df['filename'][train_test_df['filename'].isnull()]

780      NaN
1319     NaN
1560     NaN
1670     NaN
2605     NaN
2781     NaN
2914     NaN
3166     NaN
3481     NaN
3740     NaN
3741     NaN
3742     NaN
3743     NaN
3744     NaN
3745     NaN
3746     NaN
3747     NaN
3748     NaN
3749     NaN
3750     NaN
3914     NaN
4211     NaN
4365     NaN
4729     NaN
4861     NaN
5670     NaN
5674     NaN
5675     NaN
5730     NaN
5961     NaN
6265     NaN
6874     NaN
8144     NaN
8145     NaN
8150     NaN
8270     NaN
8271     NaN
8348     NaN
8403     NaN
8865     NaN
9249     NaN
9488     NaN
9492     NaN
10052    NaN
10053    NaN
10592    NaN
11141    NaN
11241    NaN
11372    NaN
11702    NaN
11703    NaN
11878    NaN
11879    NaN
12461    NaN
12462    NaN
12615    NaN
12616    NaN
13298    NaN
13958    NaN
15057    NaN
Name: filename, dtype: object

In [ ]:
#결측치 개수
train_test_df.isnull().sum()

filename      60
leye_ratio    60
reye_ratio    60
eye_angle     60
nose_ratio    60
chin_sharp    60
dtype: int64

#[]최종 feature값들 데이터프레임으로 저장

In [ ]:
train_test_df.to_csv('/content/drive/MyDrive/matlab 공모전/csv(표)/TrainTest_feature.csv', mode = 'a', index = "False")